## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 09 – Übungen

### Aufgabe 00.

Überlege dir eine Frage zum Inhalt der Vorlesung, z. B. über einen Punkt, der unklar geblieben ist oder über etwas, worüber du gerne mehr wissen möchtest.   

### Aufgabe 01. Error-Detection mithilfe der DTW-Distanz / des DTW-Scores.

In der Vorlesung haben wir die Librosa-DTW-Implementierung genutzt, um Chromagramme – und über diese Chromagramme Partituren und Aufnahmen – zu alignen.\
Das Ziel dieser Aufgabe ist es, DTW zu nutzen, um Fehler in Partituren zu erkennen.

Eine Referenz-Datei *chopin_score_reference.musicxml* enthält die Partitur des Préludes in e-Moll von Chopin.\
Unter den Dateien *chopin_score1.musicxml*, *chopin_score2.musicxml* und *chopin_score3.musicxml* sollen – mithilfe des DTW-Verfahrens – die Dateien gefunden werden, die einen Fehler enthalten.

In [2]:
import librosa
import music21 as m21
import numpy as np
import matplotlib.pyplot as plt

1) Ein paar nützliche Funktionen aus der Vorlesung:

In [3]:
def symbolic_chromagram(score, frames_per_quarter_note):

    chordified_score = score.chordify()
    # Akkord-Liste des zu analysierenden Scores
    score_chords = list(chordified_score.flatten().getElementsByClass(m21.chord.Chord))

    # Dauer des zu analysierenden Scores (in Viertelnoten ausgedrückt)
    total_duration = int(score_chords[-1].offset + score_chords[-1].quarterLength)

    # Anzahl der Analyse-Frames im Chromagramm
    number_frames_in_chromagram = total_duration*frames_per_quarter_note

    # Initialisierung des Chromagramms als 2-dimensionales Array, das nur Nullen enthält
    chromagram = np.zeros((12, number_frames_in_chromagram))

    # Iterierung über alle Akkorde in der Akkord-Liste
    for chord in score_chords:

        # Start-Frame im Chromagramm für den aktuellen Akkord
        current_chord_offset = chord.offset
        start_in_chromagram = int(current_chord_offset*frames_per_quarter_note)

        # End-Frame im Chromagramm für den aktuellen Akkord
        chord_duration = chord.quarterLength
        end_in_chromagram = int(start_in_chromagram + chord_duration*frames_per_quarter_note)

        # Bestimmung aller im aktuellen Akkord enthaltenen Pitch-Classes
        pitch_classes_current_chord = []
        for note in chord:
            pitch_class = note.pitch.pitchClass
            pitch_classes_current_chord.append(pitch_class)

        # Eintrag des aktuellen Akkords in das Chromagramm
        for time_index in np.arange(start_in_chromagram, end_in_chromagram):

            for pitch_class in pitch_classes_current_chord:
                chromagram[pitch_class, time_index] = chromagram[pitch_class, time_index] + 1
    
    return chromagram

In [4]:
def cos_distance(vector1, vector2): 

    cos_dist = 1 - (((np.dot(vector1, vector2) / (np.linalg.norm(vector1)*np.linalg.norm(vector2))) + 1)/2)

    return cos_dist

In [5]:
def cost_matrix(chromagram1, chromagram2):

    num_frames_chromagram1 = chromagram1.shape[1]
    num_frames_chromagram2 = chromagram2.shape[1]

    # Initialisierung der Cost-Matrix 
    CM = np.zeros([num_frames_chromagram1, num_frames_chromagram2])

    for i in range(num_frames_chromagram1):
        for j in range(num_frames_chromagram2):
            CM[i, j] = cos_distance(chromagram1[:, i], chromagram2[:, j])

    # Falls die Berechnung einer Cost-Matrix NaN (Not a Number) ergeben sollte, setzen wir den Wert auf 0
    CM[np.isnan(CM)] = 1

    return CM

2) Erstellung der Chromagramme aller Scores:

In [6]:
frames_per_quarter_note = 24

score_reference = m21.converter.parse('chopin_score_reference.musicxml')
chromagram_score_reference = symbolic_chromagram(score=score_reference, frames_per_quarter_note=frames_per_quarter_note)

score1 = m21.converter.parse('chopin_score1.musicxml')
chromagram_score1 = symbolic_chromagram(score=score1, frames_per_quarter_note=frames_per_quarter_note)

score2 = m21.converter.parse('chopin_score2.musicxml')
chromagram_score2 = symbolic_chromagram(score=score2, frames_per_quarter_note=frames_per_quarter_note)

score3 = m21.converter.parse('chopin_score3.musicxml')
chromagram_score3 = symbolic_chromagram(score=score3, frames_per_quarter_note=frames_per_quarter_note)

3) Schlage eine Methode vor, wie man DTW dazu nutzen kann, um zu bestimmen, welche Dateien Fehler enthalten, das heißt von der Referenz-Datei abweichende Stellen haben.

In [7]:
# Erstellung der verschiedenen Cost-Matrizen
CM1 = cost_matrix(chromagram_score1, chromagram_score_reference)
CM2 = cost_matrix(chromagram_score2, chromagram_score_reference)
CM3 = cost_matrix(chromagram_score3, chromagram_score_reference)

In [9]:
# DTW mit der ersten Test-Datei
ACM1, warping_path1 = librosa.sequence.dtw(C=CM1,
                                         step_sizes_sigma=np.array([[2,1],[1,2],[1,1]])) # Schrittbedingungen

In [10]:
# Die DTW-Distanz = der obere, rechte Eintrage in der ACM-Matrix: bestimmt, wie "gut" das gefundene DTW-Alignment ist.
# DTW-Distanz zwischen der Referenz-Datei und der ersten Test-Datei
ACM1[-1,-1]

np.float64(0.0)

In [12]:
# DTW mit der zweiten Test-Datei
ACM2, warping_path2 = librosa.sequence.dtw(C=CM2,
                                         step_sizes_sigma=np.array([[2,1],[1,2],[1,1]])) # Schrittbedingungen

In [15]:
# DTW-Distanz zwischen der Referenz-Datei und der zweiten Test-Datei
ACM2[-1,-1]

np.float64(0.0)

In [16]:
# DTW mit der dritten Test-Datei
ACM3, warping_path3 = librosa.sequence.dtw(C=CM3,
                                         step_sizes_sigma=np.array([[2,1],[1,2],[1,1]])) # Schrittbedingungen

In [17]:
# DTW-Distanz zwischen der Referenz-Datei und der dritten Test-Datei
ACM3[-1,-1]

np.float64(3.1675170953613687)

In [18]:
# Über die folgende Schleife kann bestimmt werden, ab welchem Index eine Abweichung zur Referenz-Datei auftritt
for i in range(len(warping_path3)):
    print(ACM3[warping_path3[i][0], warping_path3[i][1]])

3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953613687
3.1675170953

### Aufgabe 02. Error-Detection mithilfe der Levenshtein-Distanz.

Das Ziel dieser Aufgabe ist dasselbe wie das der vorherigen Aufgabe. Als Werkzeug soll diesmal die Levenshtein genutzt werden.

In [19]:
import librosa
import music21 as m21
import numpy as np
import matplotlib.pyplot as plt

1) Unsere Levenshtein-Distanz-Funktion aus der Vorlesung:

In [109]:
def levenshtein_distance(sequence1, sequence2):
    M = len(sequence1) + 1
    N = len(sequence2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Sequenz-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if i == 0:
                D[i, j] = j

            elif j == 0:
                D[i, j] = i

            elif sequence1[i-1] == sequence2[j-1]:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 1 

    return D[i, j]

2) Erstellung der verschiedenen Chromagramme:

In [113]:
frames_per_quarter_note = 24

score_reference = m21.converter.parse('chopin_score_reference.musicxml')
chromagram_score_reference = symbolic_chromagram(score=score_reference, frames_per_quarter_note=frames_per_quarter_note)

score1 = m21.converter.parse('chopin_score1.musicxml')
chromagram_score1 = symbolic_chromagram(score=score1, frames_per_quarter_note=frames_per_quarter_note)

score2 = m21.converter.parse('chopin_score2.musicxml')
chromagram_score2 = symbolic_chromagram(score=score2, frames_per_quarter_note=frames_per_quarter_note)

score3 = m21.converter.parse('chopin_score3.musicxml')
chromagram_score3 = symbolic_chromagram(score=score3, frames_per_quarter_note=frames_per_quarter_note)

3) Überlege dir eine Variante, bzw. eine Anwendung der Levenshtein-Distanz, um zu bestimmen, welche der Dateien Fehler enthalten. Was sagt der gefundene Score aus?

In [26]:
def levenshtein_distance_for_chromagrams(chromagram1, chromagram2):
    M = chromagram1.shape[1] + 1
    N = chromagram2.shape[1] + 1

    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if i == 0:
                D[i, j] = j

            elif j == 0:
                D[i, j] = i

            # Anwendung der Cosine-Distance, um zu bestimmen, wann Chromagramm-Frames identisch sind
            elif cos_distance(chromagram1[:, i-1], chromagram2[:, j-1]) < 0.001:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 1

    return D[i-1, j-1]

In [27]:
levenshtein_distance_for_chromagrams(chromagram_score1, chromagram_score_reference)

np.float64(0.0)

In [ ]:
levenshtein_distance_for_chromagrams(chromagram_score2, chromagram_score_reference)

In [ ]:
levenshtein_distance_for_chromagrams(chromagram_score3, chromagram_score_reference)

np.float64(24.0)

In [ ]:
# Die Levenshtein-Distanz zwischen der Referenz-Datei und der dritten Test-Datei ist 24:
# in 24 Frames unterscheiden sich also die beiden Chromagramme voneinander.
# Da frames_per_quarter_note = 24 ist, entspricht dies genau der Dauer einer Viertel-Note.